In [ ]:
# Importing the libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'
# plt.rcParams['font.family'] = 'NanumGothic'

In [ ]:
# 한글폰트 확인
pd.Series([1, 3, -5, 7]).plot(title="한글", figsize=(5, 1))

In [ ]:
# Loading dataset

train_df = pd.read_csv('/Users/kenny_jung/aiffel/data/apt/train_30.csv')
test_df = pd.read_csv('/Users/kenny_jung/aiffel/data/apt/test.csv')
park = pd.read_csv('/Users/kenny_jung/aiffel/data/apt/park.csv')
daycare = pd.read_csv('/Users/kenny_jung/aiffel/data/apt/day_care_center.csv')
hospital = pd.read_csv('/Users/kenny_jung/aiffel/data/apt/hospital.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
# 결측치 시각화
fig = plt.figure(figsize=(10, 5))
sns.heatmap(train_df.isnull(), cmap='Blues')  

In [ ]:
park.head()

In [ ]:
park.info()

In [ ]:
# 결측치 시각화
fig = plt.figure(figsize=(10, 5))
sns.heatmap(park.isnull(), cbar=False, cmap='Blues')  

In [ ]:
daycare.head()

In [ ]:
daycare.info()

In [ ]:
daycare['baby_teacher'] = daycare['day_care_baby_num'] / daycare['teacher_num']

In [ ]:
daycare['commuting_vehicle'] = daycare['is_commuting_vehicle'].map({'Y': 1, 'N': 0, '': 0, 'y': 1, 'n': 0})

In [ ]:
# 결측치 시각화
fig = plt.figure(figsize=(10, 5))
sns.heatmap(daycare.isnull(), cbar=False, cmap='Blues')  

In [ ]:
# 'city', 'gu', 'day_care_type' 별로 그룹화하고 각 그룹의 레코드 수를 계산
group = daycare.groupby(['city', 'gu', 'day_care_type']).size().reset_index(name='counts')

# 'day_care_type' 별로 컬럼을 만들고, NaN 값을 0으로 채우기
daycare_group = group.pivot_table(index=['city', 'gu'], columns='day_care_type', values='counts', fill_value=0).reset_index()

# 'day_care' total 컬럼 추가
daycare_group['total'] = daycare_group['가정'] + daycare_group['국공립'] + daycare_group['민간'] + daycare_group['법인·단체'] + daycare_group['사회복지법인'] + daycare_group['직장'] + daycare_group['협동'] 

# 컬럼 이름에 'daycare_' 추가
daycare_group.columns = ['daycare_' + col if col not in ['city', 'gu'] else col for col in daycare_group.columns]

# 컬럼 이름을 간결하게 만들기
daycare_group.columns.name = None
daycare_group = daycare_group.reset_index(drop=True)
daycare_group.head()

In [ ]:
# 'city'와 'gu' 기준으로 'baby_teacher'의 평균과 'commuting_vehicle'의 합을 계산합니다.
grouped = daycare.groupby(['city', 'gu']).agg({'baby_teacher': 'mean', 'commuting_vehicle': 'sum'}).reset_index()

# 계산된 결과를 'daycare_group'에 병합합니다.
daycare_group = pd.merge(daycare_group, grouped, on=['city', 'gu'], how='left')

# 열 이름을 변경합니다.
daycare_group.rename(columns={'baby_teacher': 'avg_baby_teacher', 'commuting_vehicle': 'sum_commuting_vehicle'}, inplace=True)

In [ ]:
daycare_group.fillna(0, inplace=True)

In [ ]:
daycare_group.sample(10)

In [ ]:
fig = plt.figure(figsize=(10, 5))
sns.countplot(data=daycare, x='city', hue='day_care_type', palette='viridis')

In [ ]:
# 'city', 'gu', 'dong', 'park_type' 별로 그룹화하고 각 그룹의 레코드 수를 계산
group = park.groupby(['city', 'gu', 'dong', 'park_type']).size().reset_index(name='counts')

# 'park_type' 별로 컬럼을 만들고, NaN 값을 0으로 채우기
park_group = group.pivot_table(index=['city', 'gu', 'dong'], columns='park_type', values='counts', fill_value=0).reset_index()

# 'park' total 컬럼 추가
park_group['total'] = park_group['근린공원'] + park_group['기타'] + park_group['도시농업공원'] + park_group['묘지공원'] + park_group['문화공원'] + park_group['소공원'] + park_group['수변공원'] + park_group['어린이공원'] + park_group['역사공원'] + park_group['체육공원']

# 컬럼 이름에 'park_' 추가
park_group.columns = ['park_' + col if col not in ['city', 'gu', 'dong'] else col for col in park_group.columns]

# 컬럼 이름을 간결하게 만들기
park_group.columns.name = None
park_group = park_group.reset_index(drop=True)
park_group.head()

In [ ]:
# 'city', 'gu'와 'dong' 기준으로 'part_area'의 합을 계산합니다.
grouped = park.groupby(['city', 'gu', 'dong']).agg({'park_area': 'sum'}).reset_index()

# 계산된 결과를 'park_group'에 병합합니다.
park_group = pd.merge(park_group, grouped, on=['city', 'gu', 'dong'], how='left')

# 열 이름을 변경합니다.
# park_group.rename(columns={'park_area': 'sum_part_area'}, inplace=True)

In [ ]:
park_group.sample(10)

In [ ]:
fig = plt.figure(figsize=(10, 5))
sns.countplot(data=park, x='city', hue='park_type', palette='viridis')

In [ ]:
city_gu_dong = pd.read_csv('/Users/kenny_jung/aiffel/datathon/aiffel_datathon/data/city_gu_dong.csv')
city_gu_dong

In [ ]:
# train_df에 'city', 'dong'을 기준으로 'gu' 추가
train_df = pd.merge(train_df, city_gu_dong[['city', 'dong', 'gu']], on=['city', 'dong'], how='left')

In [ ]:
# 'city'와 'gu'를 기준으로 'train_df'와 'daycare_group'를 병합합니다.
# 'left' 방식을 사용하여 'train_df'의 모든 행을 유지하고, 'daycare_group'에서 일치하는 행을 병합합니다.
train_df = pd.merge(train_df, daycare_group, on=['city', 'gu'], how='left')

In [ ]:
# 'city', 'gu', 'dong' 을 기준으로 'train_df'와 'park_group'를 병합합니다.
# 'left' 방식을 사용하여 'train_df'의 모든 행을 유지하고, 'park_group'에서 일치하는 행을 병합합니다.
train_df = pd.merge(train_df, park_group, on=['city', 'gu', 'dong'], how='left')

In [ ]:
# 'city', 'dong' 을 기준으로 'train_df'와 'hospital'를 병합합니다.
# 'left' 방식을 사용하여 'train_df'의 모든 행을 유지하고, 'hospital'에서 일치하는 행을 병합합니다.
train_df = pd.merge(train_df, hospital, on=['city', 'dong'], how='left')

In [ ]:
train_df.head()

In [ ]:
# 모든 열 이름을 가져옵니다.
cols = list(train_df.columns)

# 'transaction_real_price'를 제거합니다.
cols.remove('transaction_real_price')

# 'transaction_real_price'를 마지막에 추가합니다.
cols.append('transaction_real_price')

# 새로운 열 순서를 DataFrame에 적용합니다.
train_df = train_df[cols]

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.isnull().sum()

In [ ]:
# 결측치 시각화
fig = plt.figure(figsize=(10, 5))
sns.heatmap(train_df.isnull(), cbar=False, cmap='Blues')  

In [ ]:
corr = train_df.corr(numeric_only=True)
mask = np.triu(np.ones_like(corr, dtype=bool))
fig = plt.figure(figsize=(15, 13))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', linewidths=0.2, vmin = -1, vmax = 1, mask = mask)

In [ ]:
# Drop non-numeric columns
numeric_train_df = train_df.select_dtypes(include=[np.number])

# Calculate correlation
corr = numeric_train_df.corr()['transaction_real_price'].sort_values(ascending=False)

print(corr)

In [ ]:
fig = plt.figure(figsize=(10, 10))
sns.barplot(x=corr.values, y=corr.index, hue=corr.index, dodge=False, palette='coolwarm_r')

In [ ]:
# 'dong' 별로 'transaction_real_price'의 평균을 계산합니다.
avg_price_by_dong = train_df.groupby('dong')['transaction_real_price'].mean().reset_index()

# 결과를 시각화합니다.
fig = plt.figure(figsize=(10, 5))
sns.barplot(data=avg_price_by_dong, x='dong', y='transaction_real_price')
plt.xticks(rotation=90)  # x축 레이블이 겹치지 않도록 회전시킵니다.
plt.show()

In [ ]:
avg_price_by_dong.describe()

In [ ]:
train_df['transaction_real_price'].describe()

In [ ]:
fig = plt.figure(figsize=(10, 5))
sns.boxplot(data = avg_price_by_dong, x='transaction_real_price', color='skyblue')

In [ ]:
# !pip install geopy

In [ ]:
train_df.sample(10)

In [ ]:
# import pandas as pd
# from tqdm import tqdm
# from geopy.geocoders import Nominatim

# train_df = pd.read_csv('/Users/kenny_jung/aiffel/data/apt/train.csv')


# # Geopy를 사용하여 위도와 경도를 조회하는 함수
# def get_lat_lon(address):
#     geolocator = Nominatim(user_agent="kenny")
#     location = geolocator.geocode(address)
#     if location:
#         return location.latitude, location.longitude
#     else:
#         return None, None

# # 'addr_kr' 열의 각 주소에 대해 위도와 경도 조회
# # tqdm을 사용하여 진행 상황을 표시
# tqdm.pandas()
# train_df[['latitude', 'longitude']] = train_df['addr_kr'][5001:10000].progress_apply(get_lat_lon).apply(pd.Series)

# print(train_df.head())


In [ ]:
# train_df.to_csv('/Users/kenny_jung/aiffel/data/apt/train_merged.csv') 


In [ ]:
train_df.info()

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.drop(['city', 'dong', 'apt', 'gu', ], axis=1, inplace=True)

In [ ]:
# 무한대 값을 NaN으로 대체합니다.
train_df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
# 각 열의 결측치를 해당 열의 중앙값으로 채웁니다.
train_df.fillna(train_df.median(), inplace=True)

In [ ]:
train_df.isnull().sum()

In [ ]:
# train_df.to_csv('/Users/kenny_jung/aiffel/data/apt/train_merged.csv')

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_squared_log_error, mean_absolute_percentage_error

X = train_df.drop('transaction_real_price', axis=1)
y = train_df['transaction_real_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25)
model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=-1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('R2 score:', r2_score(y_test, y_pred))
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
fig = plt.figure(figsize=(10, 10))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.5)

In [ ]:
train_df.info()

In [ ]:
# 'transaction_real_price' 열에 대해 IQR을 계산합니다.
Q1 = train_df['transaction_real_price'].quantile(0.25)
Q3 = train_df['transaction_real_price'].quantile(0.75)
IQR = Q3 - Q1

# 1.5 x IQR 규칙에 따라 이상치를 제거합니다.
train_df = train_df[~((train_df['transaction_real_price'] < (Q1 - 1.5 * IQR)) | (train_df['transaction_real_price'] > (Q3 + 1.5 * IQR)))]

In [ ]:
train_df.info()

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_squared_log_error, mean_absolute_percentage_error

X = train_df.drop('transaction_real_price', axis=1)
y = train_df['transaction_real_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25)
model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=-1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('R2 score:', r2_score(y_test, y_pred))
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
fig = plt.figure(figsize=(10, 10))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.5)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression     # 선형 회귀
from sklearn.linear_model import SGDRegressor         # 확률적 경사하강법
from sklearn.tree import DecisionTreeRegressor        # 의사결정나무
from sklearn.ensemble import RandomForestRegressor    # 앙상블 모델 - 랜덤포레스트
from sklearn.svm import SVR                           # 서포트 벡터 머신
from xgboost import XGBRegressor                      # XGBoost
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_squared_log_error, mean_absolute_percentage_error

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['font.family'] = 'Avenir'

X = train_df.drop('transaction_real_price', axis=1)
y = train_df['transaction_real_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25)

MAE = [] # mean_absolute_error(true, preds)
MSE = [] # mean_squared_error(true, preds)
RMSE = [] # np.sqrt(mean_squared_error(true, preds))
MLSE = [] # mean_squared_log_error(true, preds)
RMSLE = [] # np.sqrt(mean_squared_log_error(true, preds))
R2 = [] # r2_score(true, preds)

for model in [DecisionTreeRegressor(), 
              RandomForestRegressor(n_jobs=-1), 
              # SVR(kernel='linear'), 
              # SGDRegressor(), 
              LinearRegression(n_jobs=-1),
              XGBRegressor(n_jobs=-1)
              ]:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(model.__class__.__name__, r2_score(y_test, y_pred))
    MAE.append(mean_absolute_error(y_test, y_pred))
    MSE.append(mean_squared_error(y_test, y_pred))
    RMSE.append(np.sqrt(mean_squared_error(y_test, y_pred,)))
    # MLSE.append(mean_squared_log_error(y_test, y_pred))
    # RMSLE.append(np.sqrt(mean_squared_log_error(y_test, y_pred)))
    R2.append(r2_score(y_test, y_pred))

fig, ax = plt.subplots(2, 1, figsize=(15, 8))

sns.barplot(y=R2, x=['DecisionTree', 'RandomForest', 'Linear', 'XGB'], hue = R2, palette='crest', ax=ax[0]) 
for i, R2 in enumerate(R2):
    ax[0].text(i, R2, round(R2, 4), ha='center', va='bottom')
ax[0].legend().remove()
ax[0].set_ylim([0.6, 1])
ax[0].set_title('R2 of each model')

sns.lineplot(x=['DecisionTree', 'RandomForest', 'Linear', 'XGB'], y=MAE, label='MAE', ax=ax[1], marker='o')
sns.lineplot(x=['DecisionTree', 'RandomForest', 'Linear', 'XGB'], y=RMSE, label='RMSE', ax=ax[1], marker='^', markersize=10)
# sns.lineplot(x=['DecisionTree', 'RandomForest', 'SGD', 'Linear', 'XGB'], y=RMSLE, label='RMSLE', ax=ax[1], marker='s')
ax[1].set_title('MAE/RMSE of each model')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_squared_log_error, mean_absolute_percentage_error

X = train_df.drop('transaction_real_price', axis=1)
y = train_df['transaction_real_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25)
model = RandomForestRegressor(n_jobs=-1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('R2 score:', r2_score(y_test, y_pred))
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
fig = plt.figure(figsize=(10, 10))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.5)

In [ ]:
# 모델의 하이퍼파라미터를 가져옵니다.
parameters = model.get_params()

# 하이퍼파라미터를 출력합니다.
for param, value in parameters.items():
    print(f'{param}: {value}')

In [ ]:
# from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# params={    
#     'n_estimators': [50, 100, 200],
#     # 랜덤 포레스트에서 생성할 트리의 개수를 지정합니다. 이 값이 클수록 모델은 복잡해지고, 오버피팅의 위험이 있습니다.
#     # 높은 값이 더 좋은 성능을 보장하지는 않습니다.
#     'max_depth': [None, 5, 10, 20],
#     # 각 트리의 최대 깊이를 지정합니다. 이 값이 클수록 모델은 복잡해지고, 오버피팅의 위험이 있습니다. None으로 설정하면, 노드가 모든 리프가 순수해질 때까지 확장됩니다. 
#     'min_samples_split': [0.01, 0.1, 1, 2, 5],
#     # 노드를 분할하기 위한 최소한의 샘플 데이터 수입니다. 이 값보다 적게 샘플이 있다면, 노드는 분할되지 않습니다.
#     # 높은 값은 모델을 안정적으로 만들지만, 과소적합의 위험이 있습니다.
#     'min_samples_leaf': [1, 2, 5],
#     # 리프 노드가 되기 위한 최소한의 샘플 데이터 수입니다. 이 값보다 적게 샘플이 있다면, 리프 노드는 생성되지 않습니다.
#     # 높은 값은 모델을 안정적으로 만들지만, 과소적합의 위험이 있습니다.
#     'max_features': [1, 2, 5, 10, 'auto', 'sqrt']
#     # 각 노드에서 분할에 사용할 특성의 최대 개수입니다.
#     # 'auto'로 설정하면, max_features = n_features입니다.
#     # 'sqrt'로 설정하면, max_features = sqrt(n_features)입니다.
# }

# # grid = GridSearchCV(model, param_grid=params, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
# grid = RandomizedSearchCV(model, param_distributions=params, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
# grid.fit(X_train, y_train)
# best_parameters_rf = grid.best_params_  
# best_score_rf = grid.best_score_ 
# print(best_parameters_rf)
# print(best_score_rf)

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
# from sklearn.metrics import make_scorer
# from sklearn.metrics import mean_squared_error
# import numpy as np

# # 하이퍼파라미터 그리드를 정의합니다.
# param_distributions = {
#     'n_estimators': [50, 100, 200, 300],
#     'max_features': [1.0, 0.3, 0.1],
#     'max_depth': [None, 10, 20],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'bootstrap': [True, False]
# }

# # RMSE 스코어 함수를 정의합니다.
# rmse = make_scorer(lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred)), greater_is_better=False)

# # RandomizedSearchCV를 사용하여 하이퍼파라미터 튜닝을 수행합니다.
# # random_search = RandomizedSearchCV(estimator=model, param_distributions=param_distributions, n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1, scoring=rmse)
# random_search = GridSearchCV(estimator=model, param_grid=param_distributions, cv=3, verbose=2, n_jobs=-1, scoring=rmse)
# random_search.fit(X_train, y_train)

# # 최적의 하이퍼파라미터를 출력합니다.
# print(random_search.best_params_)

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_squared_log_error, mean_absolute_percentage_error

# X = train_df.drop('transaction_real_price', axis=1)
# y = train_df['transaction_real_price']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25)
# model = RandomForestRegressor(
#                 bootstrap = True,
#                 ccp_alpha = 0.0,
#                 criterion = 'squared_error',
#                 max_depth = None,
#                 max_features = 1.0, 
#                 max_leaf_nodes = None,
#                 max_samples = None,
#                 min_impurity_decrease = 0.0,
#                 min_samples_leaf = 1,
#                 min_samples_split = 5,
#                 min_weight_fraction_leaf = 0.0,
#                 n_estimators = 100,
#                 n_jobs = -1,
#                 oob_score = False,
#                 random_state = None,
#                 verbose = 0,
#                 warm_start = False
# )
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# print('R2 score:', r2_score(y_test, y_pred))
# print('MAE:', mean_absolute_error(y_test, y_pred))
# print('MSE:', mean_squared_error(y_test, y_pred))
# print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
# fig = plt.figure(figsize=(10, 10))
# sns.scatterplot(x=y_test, y=y_pred, alpha=0.5)

In [ ]:
fig = plt.figure(figsize=(10, 5))
sns.boxplot(data = train_df, x='transaction_real_price', color='skyblue')

In [ ]:
corr = train_df.corr(numeric_only=True)
mask = np.triu(np.ones_like(corr, dtype=bool))
fig = plt.figure(figsize=(15, 13))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', linewidths=0.2, vmin = -1, vmax = 1, mask = mask)

In [ ]:
# Drop non-numeric columns
numeric_train_df = train_df.select_dtypes(include=[np.number])

# Calculate correlation
corr = numeric_train_df.corr()['transaction_real_price'].sort_values(ascending=False)

fig = plt.figure(figsize=(10, 10))
sns.barplot(x=corr.values.flatten(), y=corr.index, hue=corr.index, dodge=False, palette='coolwarm_r')


In [ ]:
import shap
shap.initjs()

In [ ]:
# Get shap values
explainer = shap.Explainer(model)
shap_values = explainer(X_test)

In [ ]:
# Absolute Mean SHAP values
shap.plots.bar(shap_values)

In [ ]:
# Beeswarm plot
shap.plots.beeswarm(shap_values)

In [ ]:
# Layered violine plot
shap.plots.violin(shap_values, plot_type = 'layered_violin')